# MintPy for signal maps

In [1]:
# Import required packages
import os
import mintpy
from pathlib import Path
from typing import List, Union
from osgeo import gdal

In [2]:
orbit_list = ['AT137']
year_list = ['2017', '2018', '2019', '2020', '2021']

## clip files to common extent

In [3]:
def get_common_overlap(file_list: List[Union[str, Path]]) -> List[float]:
    """Get the common overlap of  a list of GeoTIFF files
    
    Arg:
        file_list: a list of GeoTIFF files
    
    Returns:
         [ulx, uly, lrx, lry], the upper-left x, upper-left y, lower-right x, and lower-right y
         corner coordinates of the common overlap
    """
    
    corners = [gdal.Info(str(dem), format='json')['cornerCoordinates'] for dem in file_list]

    ulx = max(corner['upperLeft'][0] for corner in corners)
    uly = min(corner['upperLeft'][1] for corner in corners)
    lrx = min(corner['lowerRight'][0] for corner in corners)
    lry = max(corner['lowerRight'][1] for corner in corners)
    return [ulx, uly, lrx, lry]

In [4]:
def clip_hyp3_products_to_common_overlap(data_path: Union[str, Path], overlap: List[float]) -> None:
    """Clip all GeoTIFF files to their common overlap
    
    Args:
        data_dir:
            directory containing the GeoTIFF files to clip
        overlap:
            a list of the upper-left x, upper-left y, lower-right-x, and lower-tight y
            corner coordinates of the common overlap
    Returns: None
    """
    
    files_for_mintpy = ['_water_mask.tif', '_corr.tif', '_unw_phase_CNN_signal.tif', '_dem.tif', '_lv_theta.tif', '_lv_phi.tif']

    for extension in files_for_mintpy:
        print(f'working on {extension}') 
        for file in data_path.rglob(f'*{extension}'):

            dst_file = file.parent / f'{file.stem}_clipped{file.suffix}'

            gdal.Translate(destName=str(dst_file), srcDS=str(file), projWin=overlap)

## Mintpy

In [5]:
# Function to write to MintPy config file
def write_config_file(out_file, CONFIG_TXT, mode='a'): 
    """Write configuration files for MintPy to process products"""
    if not os.path.isfile(out_file) or mode == 'w':
        with open(out_file, "w") as fid:
            fid.write(CONFIG_TXT)
        print('write configuration to file: {}'.format(out_file))
    else:
        with open(out_file, "a") as fid:
            fid.write("\n" + CONFIG_TXT)
        print('add the following to file: \n{}'.format(CONFIG_TXT))

In [6]:
# function to clip files and run mintpy for multiple years 
def mintpy_multiyear(orbit_list, year_list, clip=True, mintpy=True, clean_clip=True):
    # hardcoded paths for now 
    home_path_d = '/mnt/d/indennt'
    home_path = '/mnt/c/Users/qbren/Desktop/taco/projects/indennt/proc/data'
    for orbit in orbit_list:
        for year in year_list:
            print(f'working on {orbit}, {year}')
            data_path = f'{home_path_d}/hyp3_app/{orbit}/{year}'
            mintpy_path = f'{home_path}/signal_mintpy/{orbit}/mintpy_{year}'
            mintpy_path_d = f'{home_path_d}/mintpy_app/{orbit}/'

            if clip==True:
                # identify and crop to common overlap
                print('identifying common overlap')
                dem_files = Path(data_path).glob('*/*_dem.tif')
                overlap = get_common_overlap(dem_files)
                print('clipping to common overlap')
                clip_hyp3_products_to_common_overlap(Path(data_path), overlap)

            # make output dir for mintpy
            if not os.path.exists(mintpy_path):
                os.mkdir(mintpy_path)

            # write config file for mintpy
            CONFIG_TXT = f'''# vim: set filetype=cfg:
            ##----------------------------- hyp3 ---------------------##
            mintpy.load.processor        = hyp3
            ##---------interferogram datasets:
            mintpy.load.unwFile          = {data_path}/*/*{year}*unw_phase_CNN_signal_clipped.tif
            mintpy.load.corFile          = {data_path}/*/*{year}*corr_clipped.tif
            ##---------geometry datasets:
            mintpy.load.demFile          = {data_path}/*/*{year}*dem_clipped.tif
            mintpy.load.incAngleFile     = {data_path}/*/*{year}*lv_theta_clipped.tif
            mintpy.load.waterMaskFile    = {data_path}/*/*{year}*water_mask_clipped.tif

            mintpy.deramp                = linear
            mintpy.reference.lalo        = auto
            mintpy.troposphericDelay.method  = no

            mintpy.compute.cluster    = local
            mintpy.compute.numWorker  = 6
            '''

            os.chdir(mintpy_path)
            config_file = f'{mintpy_path}/{year}_Sen{orbit}.txt'
            write_config_file(config_file, CONFIG_TXT, mode='w')

            if mintpy==True:
                # run mintpy
                print('starting mintpy')
                !smallbaselineApp.py --dir {mintpy_path} {config_file}
                print('moving outputs to drive')
                !cp -r $mintpy_path $mintpy_path_d && rm -R $mintpy_path

            if clean_clip==True:
                # remove clipped files
                print('removing clipped files')
                clipped_files = f'{data_path}/*/*_clipped.tif'
                !rm {clipped_files}

In [7]:
mintpy_multiyear(orbit_list, ['2021'], clip=True, mintpy=True, clean_clip=True)

working on AT137, 2021
identifying common overlap


/home/gbrench/sw/miniconda3/envs/mintpy/lib/python3.11/site-packages/osgeo/gdal.py:287: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


clipping to common overlap
working on _water_mask.tif
working on _corr.tif
working on _unw_phase_CNN_signal.tif
working on _dem.tif
working on _lv_theta.tif
working on _lv_phi.tif
write configuration to file: /mnt/c/Users/qbren/Desktop/taco/projects/indennt/proc/data/signal_mintpy/AT137/mintpy_2021/2021_SenAT137.txt
starting mintpy

___________________________________________________________

  /##      /## /##             /##     /#######
 | ###    /###|__/            | ##    | ##__  ##
 | ####  /#### /## /#######  /######  | ##  \ ## /##   /##
 | ## ##/## ##| ##| ##__  ##|_  ##_/  | #######/| ##  | ##
 | ##  ###| ##| ##| ##  \ ##  | ##    | ##____/ | ##  | ##
 | ##\  # | ##| ##| ##  | ##  | ## /##| ##      | ##  | ##
 | ## \/  | ##| ##| ##  | ##  |  ####/| ##      |  #######
 |__/     |__/|__/|__/  |__/   \___/  |__/       \____  ##
                                                 /##  | ##
                                                |  ######/
   Miami InSAR Time-series software